In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/coffee-bean-dataset-resized-224-x-224/Coffee Bean.csv
/kaggle/input/coffee-bean-dataset-resized-224-x-224/test/Dark/dark (2).png
/kaggle/input/coffee-bean-dataset-resized-224-x-224/test/Dark/dark (95).png
/kaggle/input/coffee-bean-dataset-resized-224-x-224/test/Dark/dark (45).png
/kaggle/input/coffee-bean-dataset-resized-224-x-224/test/Dark/dark (8).png
/kaggle/input/coffee-bean-dataset-resized-224-x-224/test/Dark/dark (83).png
/kaggle/input/coffee-bean-dataset-resized-224-x-224/test/Dark/dark (15).png
/kaggle/input/coffee-bean-dataset-resized-224-x-224/test/Dark/dark (98).png
/kaggle/input/coffee-bean-dataset-resized-224-x-224/test/Dark/dark (91).png
/kaggle/input/coffee-bean-dataset-resized-224-x-224/test/Dark/dark (22).png
/kaggle/input/coffee-bean-dataset-resized-224-x-224/test/Dark/dark (23).png
/kaggle/input/coffee-bean-dataset-resized-224-x-224/test/Dark/dark (75).png
/kaggle/input/coffee-bean-dataset-resized-224-x-224/test/Dark/dark (55).png
/kaggle/input/coffee-b

In [2]:
!pip install keras-tuner --quiet

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
import keras_tuner as kt
import os

2025-06-05 04:14:11.835675: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749096852.111842      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749096852.192501      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# Directories
train_dir = "/kaggle/input/coffee-bean-dataset-resized-224-x-224/train"
test_dir = "/kaggle/input/coffee-bean-dataset-resized-224-x-224/test"

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=(224, 224),
                                               batch_size=32,
                                               class_mode="categorical")

test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=(224, 224),
                                             batch_size=32,
                                             class_mode="categorical",
                                             shuffle=False)


Found 1200 images belonging to 4 classes.
Found 400 images belonging to 4 classes.


In [5]:
def model_builder(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Rescaling(1./255, input_shape=(224, 224, 3)))

    # Tune number of convolutional blocks
    for i in range(hp.Int("conv_blocks", 1, 3)):
        filters = hp.Choice(f"filters_{i}", values=[32, 64, 128])
        model.add(tf.keras.layers.Conv2D(filters=filters, kernel_size=(3, 3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Flatten())

    # Tune dense layer size
    units = hp.Int("dense_units", min_value=64, max_value=256, step=64)
    model.add(tf.keras.layers.Dense(units=units, activation='relu'))

    # Dropout rate tuning
    dropout_rate = hp.Float("dropout", 0.2, 0.5, step=0.1)
    model.add(tf.keras.layers.Dropout(dropout_rate))

    model.add(tf.keras.layers.Dense(train_data.num_classes, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [6]:
tuner = kt.RandomSearch(
    model_builder,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='/kaggle/working',
    project_name='coffee_tuner'
)

tuner.search(train_data, epochs=5, validation_data=test_data)

best_model = tuner.get_best_models(num_models=1)[0]


Trial 5 Complete [00h 10m 19s]
val_accuracy: 0.25

Best val_accuracy So Far: 0.5824999809265137
Total elapsed time: 00h 59m 03s


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [7]:
best_model.save("/kaggle/working/coffee_roast_best_model.h5")
best_model.save_weights("/kaggle/working/coffee_roast_best.weights.h5")


In [8]:
pred_probs = best_model.predict(test_data)
pred_classes = np.argmax(pred_probs, axis=1)
true_classes = test_data.classes
class_labels = list(test_data.class_indices.keys())

# Generate classification report
report = classification_report(true_classes, pred_classes, target_names=class_labels, output_dict=True)
report_df = pd.DataFrame(report).transpose()

# Save to CSV
report_df.to_csv("/kaggle/working/classification_report.csv")

# Display core metrics
print(report_df[['precision', 'recall', 'f1-score', 'support']])


13/13 ━━━━━━━━━━━━━━━━━━━━ 11s 826ms/step
              precision  recall  f1-score   support
Dark           0.478469  1.0000  0.647249  100.0000
Green          0.657343  0.9400  0.773663  100.0000
Light          0.808511  0.3800  0.517007  100.0000
Medium         1.000000  0.0100  0.019802  100.0000
accuracy       0.582500  0.5825  0.582500    0.5825
macro avg      0.736081  0.5825  0.489430  400.0000
weighted avg   0.736081  0.5825  0.489430  400.0000
